# vgg16

https://www.digitalocean.com/community/tutorials/popular-deep-learning-architectures-alexnet-vgg-googlenet

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=4)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

In [2]:
from vgg_nets import VGG16Plus, VGG16PlusPlus
from vgg_nets import VGG8
import datasets
import time

In [3]:
# load in data
x_train, y_train, x_val, y_val, x_test, y_test, classnames = datasets.get_dataset('cifar10', val_prop=0.2)

In [12]:
SEED = 1
np.random.seed(SEED)
tf.random.set_seed(SEED)

results = []
# all_models = ["VGG16PlusPlus, VGG16PlusPlus"]
all_models = ["VGG16Plus", "VGG16PlusPlus"]
# all_models = ["VGG15PlusPlus", "VGG15PlusPlusOffOn"]
# get dataset with validation split
x_train, y_train, x_val, y_val, x_test, y_test, classnames = datasets.get_dataset('cifar10', val_prop=0.2)

# loop through each model to train
for model_idx, model_type in enumerate(all_models):
    # clear session to free memory
    tf.keras.backend.clear_session()
    
    current_seed = SEED + model_idx
    tf.random.set_seed(current_seed)
    np.random.seed(current_seed)
    
    print(f"\n{'='*20} Training {model_type} {'='*20}")
    
    if model_type == "VGG4Plus":
        model = VGG4Plus(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG15":
        model = VGG15(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG15Plus":
        model = VGG15Plus(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG15PlusPlus":
        model = VGG15PlusPlus(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG15PlusPlus":
        model = VGG15PlusPlus(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG15PlusPlusOffOn":
        model = VGG15PlusPlusOffOn(C=10, input_feats_shape=(32, 32, 3), wt_init='he')
    elif model_type == "VGG16Plus":
        model = VGG16Plus(C=10, input_feats_shape=(32, 32, 3), wt_init="he")
        # print("hii!")
    elif model_type == "VGG16PlusPlus":
        model = VGG16PlusPlus(C=10, input_feats_shape=(32, 32, 3), wt_init="he")
        # print(model)
    
    # compile with AdamW optimizer
    print(model_type)
    model.compile(optimizer='adamw')
    
    # train the model
    start_time = time.time()
    print(f"Starting training for {model_type}...")
    
    train_loss_hist, val_loss_hist, val_acc_hist, epochs = model.fit(
        x_train, y_train, 
        x_val, y_val, 
        max_epochs=100,
        patience=15,
        lr_patience=4,
        verbose=True,
        lr_decay_factor=0.5,
        lr_max_decays=12
    )
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
    
    # evaluate on test set
    test_acc, test_loss = model.evaluate(x_test, y_test)
    print(f"{model_type} Test Accuracy: {test_acc:.4f}")
    
    result = {
        'model_type': model_type,
        'test_accuracy': test_acc,
        'test_loss': test_loss,
        'train_loss_history': train_loss_hist,
        'val_loss_history': val_loss_hist,
        'val_acc_history': val_acc_hist,
        'epochs': epochs,
        'training_time': training_time
    }
    results.append(result)
    
    # Save individual results in case notebook crashes
    np.save(f"{model_type}_results_10b.npy", result)
    print(f"Saved results for {model_type}")



==================== Training VGG16Plus ====================
VGG16Plus
---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
dense_block:
	Dropout layer output(dense_block/dropout_layer_1) shape: [1, 4096]
	Dense layer output(dense_block/dense_layer_1) shape: [1, 4096]
	Dropout layer output(dense_block/dropout_layer_0) shape: [1, 4096]
	Dense layer output(dense_block/dense_layer_0) shape: [1, 4096]
Flatten layer output(flatten) shape: [1, 512]
conv_block_5:
	MaxPool2D layer output(conv_block_5/max_pool_layer_2) shape: [1, 1, 1, 512]
	Conv2D layer output(conv_block_5/conv_layer_2) shape: [1, 2, 2, 512]
	Conv2D layer output(conv_block_5/conv_layer_1) shape: [1, 2, 2, 512]
	Conv2D layer output(conv_block_5/conv_layer_0) shape: [1, 2, 2, 512]
conv_block_4:
	MaxPool2D layer output(conv_block_4/max_pool_layer_2) shape: [1, 2, 2, 512]
	Conv2D layer output(conv_block_4/conv_layer_2) shape: [1, 4, 4, 512]
	Conv2D layer output(conv

# AlexAndDerekNet

https://medium.com/codex/alexnet-complete-architecture-dc3a9920cdd

In [21]:
from vgg_nets import VGG8OnOff, VGG8OnOffNoReduction

In [ ]:
# Set random seed and clear session
tf.random.set_seed(0)
tf.keras.backend.clear_session()

# Create network - changed to VGG8
model = VGG8(C=10, 
            input_feats_shape=x_train.shape[1:], 
            wt_init="he",
            reg=0.6,
            conv_dropout=False)

# Set loss function and compile model
model.compile(loss='cross_entropy', optimizer="adamw", lr=0.001)

print(f"Starting training with patience={4}...")

start_total = time.time()

# Train the model
train_loss_hist, val_loss_hist, val_acc_hist, e = model.fit(
    x_train, y_train, x_val, y_val, patience=4)

total_time = time.time() - start_total

# Evaluate on test set
test_acc, test_loss = model.evaluate(x_test, y_test, batch_sz=128)


---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
dense_block:
	Dropout layer output(dense_block/dropout_layer_0) shape: [1, 512]
	Dense layer output(dense_block/dense_layer_0) shape: [1, 512]
Flatten layer output(flatten) shape: [1, 4096]
conv_block_3:
	MaxPool2D layer output(conv_block_3/max_pool_layer_1) shape: [1, 4, 4, 256]
	Conv2D layer output(conv_block_3/conv_layer_1) shape: [1, 8, 8, 256]
	Conv2D layer output(conv_block_3/conv_layer_0) shape: [1, 8, 8, 256]
conv_block_2:
	MaxPool2D layer output(conv_block_2/max_pool_layer_1) shape: [1, 8, 8, 128]
	Conv2D layer output(conv_block_2/conv_layer_1) shape: [1, 16, 16, 128]
	Conv2D layer output(conv_block_2/conv_layer_0) shape: [1, 16, 16, 128]
conv_block_1:
	MaxPool2D layer output(conv_block_1/max_pool_layer_1) shape: [1, 16, 16, 64]
	Conv2D layer output(conv_block_1/conv_layer_1) shape: [1, 32, 32, 64]
	Conv2D layer output(conv_block_1/conv_layer_0) shape: [1, 32,

In [17]:
# Set random seed and clear session
tf.random.set_seed(0)
tf.keras.backend.clear_session()

# Create network - changed to VGG8
model = VGG8(C=10, 
            input_feats_shape=x_train.shape[1:], 
            wt_init="he",
            reg=0.6,
            conv_dropout=True)

# Set loss function and compile model
model.compile(loss='cross_entropy', optimizer="adamw", lr=0.001)

print(f"Starting training with patience={4}...")

start_total = time.time()

# Train the model
train_loss_hist, val_loss_hist, val_acc_hist, e = model.fit(
    x_train, y_train, x_val, y_val, patience=4)

total_time = time.time() - start_total

# Evaluate on test set
test_acc, test_loss = model.evaluate(x_test, y_test, batch_sz=128)

---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
dense_block:
	Dropout layer output(dense_block/dropout_layer_0) shape: [1, 512]
	Dense layer output(dense_block/dense_layer_0) shape: [1, 512]
Flatten layer output(flatten) shape: [1, 4096]
conv_block_3:
	Dropout layer output(conv_block_3/dropout_layer_1) shape: [1, 4, 4, 256]
	MaxPool2D layer output(conv_block_3/max_pool_layer_1) shape: [1, 4, 4, 256]
	Conv2D layer output(conv_block_3/conv_layer_1) shape: [1, 8, 8, 256]
	Conv2D layer output(conv_block_3/conv_layer_0) shape: [1, 8, 8, 256]
conv_block_2:
	Dropout layer output(conv_block_2/dropout_layer_1) shape: [1, 8, 8, 128]
	MaxPool2D layer output(conv_block_2/max_pool_layer_1) shape: [1, 8, 8, 128]
	Conv2D layer output(conv_block_2/conv_layer_1) shape: [1, 16, 16, 128]
	Conv2D layer output(conv_block_2/conv_layer_0) shape: [1, 16, 16, 128]
conv_block_1:
	Dropout layer output(conv_block_1/dropout_layer_1) shape: [1, 1

In [18]:
print(f"Test acc: {test_acc}")

Test acc: 0.7931690812110901


In [19]:
# Set random seed and clear session
tf.random.set_seed(0)
tf.keras.backend.clear_session()

model = VGG8OnOff(C=10, 
            input_feats_shape=x_train.shape[1:], 
            wt_init="he",
            reg=0.6,
            conv_dropout=True)

# Set loss function and compile model
model.compile(loss='cross_entropy', optimizer="adamw", lr=0.001)

print(f"Starting training with patience={10}...")

start_total = time.time()

# Train the model
train_loss_hist, val_loss_hist, val_acc_hist, e = model.fit(
    x_train, y_train, x_val, y_val, patience=10)

total_time = time.time() - start_total

# Evaluate on test set
test_acc, test_loss = model.evaluate(x_test, y_test, batch_sz=128)
print(total_time)
print("test_acc")

---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
dense_block:
	Dropout layer output(dense_block/dropout_layer_0) shape: [1, 512]
	Dense layer output(dense_block/dense_layer_0) shape: [1, 512]
Flatten layer output(flatten) shape: [1, 1024]
conv_block_3:
	Dropout layer output(conv_block_3/dropout_layer_1) shape: [1, 2, 2, 256]
	MaxPool2D layer output(conv_block_3/max_pool_layer_1) shape: [1, 2, 2, 256]
	Conv2D layer output(conv_block_3/conv_layer_1) shape: [1, 4, 4, 256]
	Conv2D layer output(conv_block_3/conv_layer_0) shape: [1, 4, 4, 256]
conv_block_2:
	Dropout layer output(conv_block_2/dropout_layer_1) shape: [1, 4, 4, 128]
	MaxPool2D layer output(conv_block_2/max_pool_layer_1) shape: [1, 4, 4, 128]
	Conv2D layer output(conv_block_2/conv_layer_1) shape: [1, 8, 8, 128]
	Conv2D layer output(conv_block_2/conv_layer_0) shape: [1, 8, 8, 128]
conv_block_1:
	Dropout layer output(conv_block_1/dropout_layer_1) shape: [1, 8, 8,

In [20]:
print(f"Test acc: {test_acc}")

Test acc: 0.7519030570983887


In [22]:
# Set random seed and clear session
tf.random.set_seed(0)
tf.keras.backend.clear_session()

model = VGG8OnOffNoReduction(C=10, 
            input_feats_shape=x_train.shape[1:], 
            wt_init="he",
            reg=0.6,
            conv_dropout=True)

# Set loss function and compile model
model.compile(loss='cross_entropy', optimizer="adamw", lr=0.001)

print(f"Starting training with patience={10}...")

start_total = time.time()

# Train the model
train_loss_hist, val_loss_hist, val_acc_hist, e = model.fit(
    x_train, y_train, x_val, y_val, patience=10)

total_time = time.time() - start_total

# Evaluate on test set
test_acc, test_loss = model.evaluate(x_test, y_test, batch_sz=128)
print(total_time)
print("test_acc")

---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
dense_block:
	Dropout layer output(dense_block/dropout_layer_0) shape: [1, 512]
	Dense layer output(dense_block/dense_layer_0) shape: [1, 512]
Flatten layer output(flatten) shape: [1, 4096]
conv_block_3:
	Dropout layer output(conv_block_3/dropout_layer_1) shape: [1, 4, 4, 256]
	MaxPool2D layer output(conv_block_3/max_pool_layer_1) shape: [1, 4, 4, 256]
	Conv2D layer output(conv_block_3/conv_layer_1) shape: [1, 8, 8, 256]
	Conv2D layer output(conv_block_3/conv_layer_0) shape: [1, 8, 8, 256]
conv_block_2:
	Dropout layer output(conv_block_2/dropout_layer_1) shape: [1, 8, 8, 128]
	MaxPool2D layer output(conv_block_2/max_pool_layer_1) shape: [1, 8, 8, 128]
	Conv2D layer output(conv_block_2/conv_layer_1) shape: [1, 16, 16, 128]
	Conv2D layer output(conv_block_2/conv_layer_0) shape: [1, 16, 16, 128]
conv_block_1:
	Dropout layer output(conv_block_1/dropout_layer_1) shape: [1, 1

In [23]:
print(f"Test acc: {test_acc}")

Test acc: 0.8079928159713745
